In [1]:
pwd

u'E:\\Github\\ipython.notebook.dir'

In [2]:
%pylab
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix
tf.qinv = tf.quaternion_inverse
tf.qmul = tf.quaternion_multiply

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [289]:
'''
float sdf_orig_host;
float cos_host;
float sdf_cos_host;
float tdist_real_host;
bool snorm_oppo_host;
bool is_curr_edge_host;
bool is_non_edge_near0_host;
short depth_prev_host;
short diff_cp_host;
'''

'\nfloat sdf_orig_host;\nfloat cos_host;\nfloat sdf_cos_host;\nfloat tdist_real_host;\nbool snorm_oppo_host;\nbool is_curr_edge_host;\nbool is_non_edge_near0_host;\nshort depth_prev_host;\nshort diff_cp_host;\n'

In [80]:
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log.txt'
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[106,35,176].txt' #折痕边的对边, 顶棱被腐蚀
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[106,35,176]-tv2.txt' #由 tsdf-orig 原始策略得到的
# fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[106,35,176]-tv2-tdreal.txt' #折痕边的对边, 顶棱被腐蚀
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[177,39,121].txt' #折痕短边顶棱, 被腐蚀
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[173,54,121].txt' #折痕短边外侧表面, 开始在外面, 后来成了内部
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[164,55,115].txt' #折痕短边内侧表面,
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[145,54,94].txt' #折痕短边左侧长边，破坏穿透

fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[145,54,94]-2.txt' #前面 9列, 增加 depth_curr, 为10列, 且 sdf-cos >= -tranc_dist 判定
# fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[145,54,94]-3.txt' #10列, 且 sdf-cos >= -td-real 判定
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[108,34,177].txt' 
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[118,45,112].txt' #椅背左侧, 面上, 外面, 随便一内点
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kf-dbg-log=[136,58,190].txt' #靠近内面的 表面内部, 观察 v19 是否有效
dat = np.loadtxt(fn, delimiter=',')
# plot(dat[:,0])

# sdf = dat[:,0]
fid_cut = 431
# cols = 9
# sdf, cos, sdf_cos, td_real, sn_oppo, is_curr_edge, is_non_edge_near0, depth_prev, diff_cp = np.hsplit(dat, cols) #abs(dat[:,0]) > 1e-2
# sdf_1, cos_1, sdf_cos_1, td_real_1, sn_oppo_1, is_curr_edge_1, is_non_edge_near0_1, depth_prev_1, diff_cp_1 = np.hsplit(dat[:fid_cut], cols)

cols = 10
sdf, cos, sdf_cos, td_real, sn_oppo, is_curr_edge, is_non_edge_near0, depth_curr, depth_prev, diff_cp = np.hsplit(dat, cols) #abs(dat[:,0]) > 1e-2
sdf_1, cos_1, sdf_cos_1, td_real_1, sn_oppo_1, is_curr_edge_1, is_non_edge_near0_1, depth_curr_1, depth_prev_1, diff_cp_1 = np.hsplit(dat[:fid_cut], cols)
sdf_cos = sdf * cos
sdf_cos_1 = sdf_cos[:fid_cut]

# sdf1 = sdf[:fid_cut] #f0~430, 1st view point
# sdf[abs(sdf) > 1e-2] = 0

def mainPlot():
    plot(sdf)
    plot(sdf_cos) #之前数据可能错, 不要直接 plot
    #plot(sdf * cos)
    plot( -td_real )
    plot( (sn_oppo - 0.5) * 8e-3 )
    ylim(-0.01, +0.01)
    grid(True)
    pass


close()
mainPlot()
tight_layout()

In [56]:
'''
调试绘制 subplot 51X
'''
def padYlim():
    ymin, ymax = ylim()
    ydt = ymax - ymin
    ylim(ymin - ydt/10., ymax + ydt/10. )
    pass
    
# close()
figure()

ax1 = subplot(5,1,1)
title('sdf & sdf_cos')
# plot(sdf)
# plot(sdf_cos)
# plot( -td_real )
mainPlot()
grid(True)

subplot(5,1,2, sharex=ax1)
title('cos')
plot(cos)

subplot(5,1,3, sharex=ax1)
title('td_real')
plot( td_real )
padYlim()

subplot(5,1,4, sharex=ax1)
title('sn_oppo')
plot( sn_oppo )
padYlim()
# grid(True)

is_curr_edge, is_non_edge_near0
subplot(5,1,5, sharex=ax1)
title('is_curr_edge & is_non_edge_near0')
plot( is_curr_edge )
plot( is_non_edge_near0 )
legend(list('cp'))
padYlim()


tight_layout()

In [85]:
MAX_WEIGHT = 256
tdist = 0.005

tsdf2_vec = []
switch_flag_vec = [False] * len(dat) #tsdf1 & 2 若交换, 标记 True
t2_flag_vec = [False] * len(dat)

valid_flag_vec = [True] * len(dat)

def calc_tsdf(dat, ver_):
    print 'ver_:', ver_

    #global tsdf_p, weight_p, tsdf2_p, weight2_p#, tsdf_vec, tsdf2_vec
    tsdf_p, weight_p, wrk,  = 0, 0, 1, 
    tsdf2_p, weight2_p = 0, 0

    
    tsdf_vec = [] #待返回值
    for i, frame in enumerate( dat[:]):
        #print i, frame
        sdf_i, cos_i, sdf_cos_i, td_real_i, sn_oppo_i, \
        is_curr_edge_i, is_non_edge_near0_i, depth_curr_i, depth_prev_i, diff_cp_i = frame
        #is_curr_edge_i, is_non_edge_near0_i, depth_prev_i, diff_cp_i = frame

        tsdf_vec.append(tsdf_p) #放最前面, 后面更新 [-1] 即可

        wrk = 1 #必须局部变量

        '原始 tsdf-fusion 算法'
        if ver_ == 1:
            print 'i:=', i, 
            if sdf_i < -tdist: #orig
        #     if sdf_i < -td_real_i: #因大部分时候 sdf < -td_real, 此判定导致 sdf_avg 一直向正值漂移
    #         if sdf_cos_i < -td_real_i: #用 sdf_cos 判定
                print 'sdf_i < -tdist'

                valid_flag_vec[i] = False
                tsdf_vec.append( tsdf_vec[-1] if tsdf_vec else 0 ) #防止 vec=[]空的
                continue

            else:
                print 'sdf_i ok'
                tsdf_i = min(1, sdf_i / tdist) #float tsdf_curr = fmin (1.0f, sdf * tranc_dist_inv);
            #     tsdf_i = min(1, sdf_cos_i / tdist)

                tsdf_p = (tsdf_p * weight_p + tsdf_i * wrk) / (weight_p + wrk)
                weight_p = weight_p + wrk
        #         weight_p = min(weight_p + wrk, MAX_WEIGHT)

        'tsdf-v19.x 方案'
        if int(ver_) == 19: #ver可以:=19.x
            print 'i=', i, 'sdf_i, sdf_cos_i, tdist, td_real_i:', sdf_i, sdf_cos_i, tdist, td_real_i, 'd_curr, w1p:', depth_curr_i, weight_p

            if ver_ == 19.5:
                print 'if ver_ == 19.5'
                sdf_use = sdf_cos_i
                td_use = tdist * cos_i
                
            #elif ver_ == 19.6:
            elif int(ver_*10) == 196:
                print 'elif ver_ == 19.6'
                if not is_non_edge_near0_i: #//必然一直是 edge
                    sdf_use = sdf_cos_i
                    td_use = tdist * cos_i
                else: #//确认非contour
                    if is_curr_edge_i: #//但当前是edge
                        td_use = tdist * cos_i
                    else: #//且当前在内部
                        if ver_ == 19.64:
                            print 'if ver_ == 19.64'
#                             if sdf_cos_i < 0:
#                                 sdf_use = sdf_cos_i
                            sdf_use = sdf_i if sdf_i > 0 else sdf_cos_i
                        else:
                            print 'if ver_ != 19.64'            
                            sdf_use = sdf_cos_i
                        td_use = tdist
                        pass
                    pass
                pass
            
    #         if depth_curr_i != 0 and sdf_cos_i >= -td_real_i:
#             if depth_curr_i != 0 and sdf_cos_i >= -tdist * cos_i:
    #         if depth_curr_i != 0 and sdf_i >= -tdist:
            if depth_curr_i != 0 and sdf_use >= -td_use:
                tsdf_i = min(1, sdf_use / tdist)
#                 print '\tsdf_i >= -tdist:', sdf_i, tdist
                print '\tsdf_i >= -tdist:', sdf_use, td_use, 'sdf, sdf_cos:', sdf_i, sdf_cos_i, 'tsdf_i:', tsdf_i
    #             tsdf_i = min(1, sdf_i / tdist)
                diff_cp_th = 20 #depth-diff-curr-prev #20mm
                if is_non_edge_near0_i and is_curr_edge_i \
                and depth_prev_i and diff_cp_i > diff_cp_th: #注意是 p非c
                    wrk = 0 #应该  continue, 
                    print '>diff_cp_th, i=', i
                    continue


                if tsdf_p + tsdf_i < 0 \
                and is_non_edge_near0_i and not is_curr_edge_i:
                    print '\ttsdf_p + tsdf_i < 0: p, p2, c, w1, w2=', tsdf_p, tsdf2_p, tsdf_i, weight_p, weight2_p

                    '异侧视角'
                    if sn_oppo_i:
                        t2_flag_vec[i] = True

    #                     print
                        if 0 == weight2_p:
                            tsdf2_p = tsdf_i
                            weight2_p = wrk
                        else:
                            if abs( tsdf_i - tsdf_p) < abs( tsdf_i - tsdf2_p):
                                tsdf_p = (tsdf_p * weight_p + tsdf_i * wrk) / (weight_p + wrk)
                                #weight_p = weight_p + wrk
                                weight_p = min(weight_p + wrk, MAX_WEIGHT)

                            else:
                                tsdf2_p = (tsdf2_p * weight2_p + tsdf_i * wrk) / (weight2_p + wrk)
                                #weight2_p = weight2_p + wrk
                                weight2_p = min(weight2_p + wrk, MAX_WEIGHT)

                                if weight2_p > weight_p: #交换 1 & 2
                                    weight_p, weight2_p = weight2_p, weight_p
                                    tsdf_p, tsdf2_p = tsdf2_p, tsdf_p
                                    switch_flag_vec[i] = True                        
                        continue

                tsdf_p = (tsdf_p * weight_p + tsdf_i * wrk) / (weight_p + wrk)
                #weight_p = weight_p + wrk
                weight_p = min(weight_p + wrk, MAX_WEIGHT)
                print '\ttsdf_p, weight_p, tsdf_i, wrk:', tsdf_p, weight_p, tsdf_i, wrk

                if weight2_p > 0:
                    print '\tw2-=1'
                    weight2_p -=1
                    if 0 == weight2_p:
                        tsdf2_p = 0

#             elif depth_curr_i != 0 and sdf_cos_i > -4*tdist \
            elif depth_curr_i != 0 and sdf_use > -4*td_use \
            and is_non_edge_near0_i and not is_curr_edge_i:
                #print '\tsdf_cos_i > -4*tdist:', sdf_cos_i, tdist
                print '\tsdf_use > -4*td_use:', sdf_use, td_use
                POS_VALID_WEIGHT_TH = 30
                if tsdf_p > 0 and weight_p < POS_VALID_WEIGHT_TH:
                    weight_p = max(0, weight_p -1)
                    if 0 == weight_p:
                        tsdf_p = 0

        #tsdf_vec.append(tsdf_p) #放这里错了, 放最前面
        tsdf_vec[-1] = tsdf_p
#         if i >= 833:
        print '\ti:=', i, 'tsdf_p:', tsdf_p, 'len(tsdf_vec):', len(tsdf_vec)

    tsdf_vec = array(tsdf_vec)
    return tsdf_vec

# tsdf_vec = calc_tsdf(dat, 1)
# tsdf_vec = calc_tsdf(dat, 19.5)
tsdf_vec = calc_tsdf(dat, 19.6)
# tsdf_vec = calc_tsdf(dat, 19.64)
tsdf_vec_scale = tsdf_vec * 0.01 #缩放到 mainPlot-ylim, 以便画在一起
sdf_vec_avg = tsdf_vec * tdist

# close()
mainPlot()
plot(sdf_vec_avg)
plot(tsdf_vec_scale)
legend(['sdf', 'sdf_cos', 'td_real', 'sn_oppo', 'sdf_vec_avg', 'tsdf_vec_scale'])
# plot( valid_flag_vec) #看着太乱

print 'switch 1&2: %d times' % sum(switch_flag_vec)

ver_: 19.6
i= 0 sdf_i, sdf_cos_i, tdist, td_real_i: 0.00108284 0.000774314 0.005 0.00357539 d_curr, w1p: 945.0 0
elif ver_ == 19.6
if ver_ != 19.64
	sdf_i >= -tdist: 0.000774314 0.005 sdf, sdf_cos: 0.00108284 0.000774314 tsdf_i: 0.1548628
	tsdf_p, weight_p, tsdf_i, wrk: 0.1548628 1 0.1548628 1
	i:= 0 tsdf_p: 0.1548628 len(tsdf_vec): 1
i= 1 sdf_i, sdf_cos_i, tdist, td_real_i: -0.00285643 -0.00194491 0.005 0.005 d_curr, w1p: 941.0 1
elif ver_ == 19.6
if ver_ != 19.64
	sdf_i >= -tdist: -0.00194491 0.005 sdf, sdf_cos: -0.00285643 -0.00194491 tsdf_i: -0.388982
	tsdf_p + tsdf_i < 0: p, p2, c, w1, w2= 0.1548628 0 -0.388982 1 0
	tsdf_p, weight_p, tsdf_i, wrk: -0.1170596 2 -0.388982 1
	i:= 1 tsdf_p: -0.1170596 len(tsdf_vec): 2
i= 2 sdf_i, sdf_cos_i, tdist, td_real_i: 0.00106585 0.00106585 0.005 0.005 d_curr, w1p: 945.0 2
elif ver_ == 19.6
if ver_ != 19.64
	sdf_i >= -tdist: 0.00106585 0.005 sdf, sdf_cos: 0.00106585 0.00106585 tsdf_i: 0.21317
	tsdf_p, weight_p, tsdf_i, wrk: -0.00698306666667 3 0.

In [76]:
cos[:100],

(array([[ 0.715079],
        [ 0.680887],
        [ 0.732073],
        [ 0.715954],
        [ 0.647652],
        [ 0.677595],
        [ 0.67476 ],
        [ 0.699044],
        [ 0.696538],
        [ 0.722977],
        [ 0.707903],
        [ 0.709569],
        [ 0.716472],
        [ 0.703172],
        [ 0.721841],
        [ 0.708909],
        [ 0.7146  ],
        [ 0.721541],
        [ 0.710438],
        [ 0.689354],
        [ 0.698838],
        [ 0.72424 ],
        [ 0.7148  ],
        [ 0.718937],
        [ 0.735667],
        [ 0.737978],
        [ 0.743025],
        [ 0.772988],
        [ 0.760016],
        [ 0.770201],
        [ 0.750056],
        [ 0.767674],
        [ 0.75397 ],
        [ 0.768225],
        [ 0.729143],
        [ 0.732792],
        [ 0.744896],
        [ 0.74343 ],
        [ 0.747018],
        [ 0.757573],
        [ 0.761595],
        [ 0.772012],
        [ 0.763382],
        [ 0.775475],
        [ 0.757376],
        [ 0.761284],
        [ 0.764116],
        [ 0.8

In [79]:
zip(range(100), sdf[:100], sdf_cos[:100])

[(0, array([ 0.00108284]), array([ 0.00077431])),
 (1, array([-0.00285643]), array([-0.00194491])),
 (2, array([ 0.00106585]), array([ 0.00106585])),
 (3, array([ 0.00147754]), array([ 0.00147754])),
 (4, array([-0.00364864]), array([-0.00236305])),
 (5, array([-0.00105995]), array([-0.00071822])),
 (6, array([-0.00133193]), array([-0.00089873])),
 (7, array([-0.00149304]), array([-0.0010437])),
 (8, array([-0.00194275]), array([-0.0013532])),
 (9, array([-0.0018096]), array([-0.0013083])),
 (10, array([-0.0018236]), array([-0.00129093])),
 (11, array([-0.00173938]), array([-0.00123421])),
 (12, array([-0.00166714]), array([-0.00119446])),
 (13, array([-0.00105166]), array([-0.0007395])),
 (14, array([-0.00104111]), array([-0.00075152])),
 (15, array([-0.00107539]), array([-0.00076235])),
 (16, array([-0.00108618]), array([-0.00077618])),
 (17, array([-0.00113612]), array([-0.00081976])),
 (18, array([-0.00377697]), array([-0.0026833])),
 (19, array([-0.00186467]), array([-0.00128542])

In [23]:
tsdf_vec[0:11]

array([ 0.1548628 , -0.1170596 , -0.00698307,  0.0686397 , -0.03961024,
       -0.05694907, -0.07449149, -0.09127255, -0.11120227, -0.12624804,
       -0.1382424 ])

In [11]:
tsdf_vec[822:844]

array([-0.25215676, -0.25215676, -0.25215676, -0.2483979 , -0.24389258,
       -0.24056201, -0.23666932, -0.23277213, -0.22793908, -0.22583436,
       -0.22017588, -0.21601562, -0.21292774, -0.21141916, -0.20844783,
       -0.20466304, -0.20209032, -0.20074131, -0.19741737, -0.19438894,
       -0.19264912, -0.19059953])

In [25]:
figure()
plot(tsdf_vec)

In [50]:
1/3. is 1/3.

False

In [51]:
1.4 is 1.4

True

In [52]:
1e3 is 1e3

True

In [55]:
1111is 1111

True

In [284]:
dat[1408]

array([  3.24547000e-03,   8.98406000e-01,   2.91575000e-03,
         1.50000000e-03,   0.00000000e+00,   1.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   7.32000000e+02])

In [288]:
n=1408
depth_prev[n], sn_oppo[n], sdf[n]

(array([ 0.]), array([ 0.]), array([ 0.00324547]))

In [209]:
len( switch_flag_vec)

1935

In [264]:
figure()
title('t2_flag_vec')
plot(t2_flag_vec)

In [275]:
plot( switch_flag_vec)

In [182]:
len(sdf_vec_avg)

1935

In [184]:
td_real[0]

array([ 0.00213396])

In [191]:
sdf[0]

array([-0.00210637])

In [186]:
dat[0]

array([ -2.10637000e-03,   7.42495000e-01,  -1.56397000e-03,
         2.13396000e-03,   0.00000000e+00,   1.00000000e+00,
         0.00000000e+00,   8.49000000e+02,  -1.00000000e+00])

In [189]:
sdf_vec_avg[0]

0.00049068600000000005

In [171]:
plot(tsdf_vec)


In [17]:
mu = sdf1.mean()
sigma = sdf1.std()

-0.00052336167541860458

In [21]:
sqrt(sdf1.var())

0.0014267059015513012

In [22]:
sdf1.std()

0.0014267059015513012

In [9]:
dat[431]

array([ -2.11591000e-01,   1.50000000e-03,   0.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   7.48000000e+02,
        -7.48000000e+02])